# EE499 Final Project
# Premier League Predictor
Hojeong Lee and Nate Chism

In [32]:
import numpy as np
import autograd.numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from autograd import grad 
from autograd import hessian
import math
import copy
import pandas as pd
import sys
import time
from IPython.display import clear_output
readDataPath = './readData/'
sys.path.append('./')

from sklearn.linear_model import Perceptron

In [46]:
season_data = pd.read_csv('./readData/df_full_premierleague.csv')
season_csv_to_array = np.array(season_data)
# print(range(0, len(house_csv_to_array) -1))
print(season_csv_to_array.shape)
print(season_csv_to_array)

(4070, 114)
[[0 'https://www.premierleague.com/match/7186' '10/11' ... 0.78 -4.0 55.6]
 [1 'https://www.premierleague.com/match/7404' '10/11' ... 0.32 17.0 60.2]
 [2 'https://www.premierleague.com/match/7255' '10/11' ... 0.38 9.0 66.7]
 ...
 [4067 'https://www.premierleague.com/match/59178' '20/21' ... 0.69 4.0
  64.1]
 [4068 'https://www.premierleague.com/match/59182' '20/21' ... 0.42 6.0
  54.2]
 [4069 'https://www.premierleague.com/match/59052' '20/21' ... 0.87 5.0
  71.1]]


In [47]:
# 2, 4, 5, 6, 8 - 31, 38 - 49, 56  
print(season_csv_to_array[5][100])

nan


In [48]:
i, j = season_csv_to_array.shape
print(i)
print(j)
match_target_scores = np.zeros((i,3))
# match_teams = np.array((i, 3))
for it_1 in range(0, i):
    # store game unique id
    match_target_scores[it_1][0] = season_csv_to_array[it_1][0]
    #store game unique id, home team, away team
    # match_teams[it_1][0] = season_csv_to_array[it_1][0]
    # match_teams[it_1][1] = season_csv_to_array[it_1][4]
    # match_teams[it_1][2] = season_csv_to_array[it_1][5]
    if season_csv_to_array[it_1][34] > 0:
        match_target_scores[it_1][1] = 1
    elif season_csv_to_array[it_1][34] < 0:
        match_target_scores[it_1][1] = -1
    else:
        match_target_scores[it_1][1] = 0

print(match_target_scores[678])

4070
114
[678.  -1.   0.]


In [50]:
season_data_cleaned = np.zeros((i, 102))

# print(season_csv_to_array[5])
season_csv_to_array[pd.isnull(season_csv_to_array)] = 0
# print(season_csv_to_array[5])
# 2, 4, 5, 6, 8 - 31, 38 - 49, 56  
for it in range(0, i):
    ct = 0
    for it2 in range(0, j):
        
        # if(np.isnan(season_csv_to_array[it][it2])):
        #     print(season_csv_to_array[it][it2])

        if it2 not in [1, 2, 3, 4, 5, 6, 7, 32, 33, 34, 35, 36, 37]:
            season_data_cleaned[it][ct] = season_csv_to_array[it][it2]
            # print(ct)
            ct = ct + 1

        elif it2 == 2:
            season_number = (int)(season_csv_to_array[it][it2][0] + season_csv_to_array[it][it2][1])
            season_data_cleaned[it][ct] = season_number
            ct = ct + 1

        
        

# print(season_data_cleaned)
print(season_data_cleaned[5])


[  5.   10.   15.    5.    9.    4.  609.   65.6   0.   19.    4.   23.
 820.    2.   32.    3.    7.    0.  313.   34.4   0.    9.    4.   21.
 527.    2.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0. ]


In [7]:
print("it: ", it, " ct: ", ct, " it2: ", it2)

it:  4069  ct:  102  it2:  113


In [8]:
print(season_data_cleaned[4000][1])

20.0


In [15]:
season_data_cleaned[123][0]

123.0

In [22]:
seasons_10to18 = np.zeros((3419, 102))
season_19 = np.zeros((380, 102))
season_20 = np.zeros((271, 102))

Y_train = np.zeros((3419,))
Y_test1 = np.zeros((380,))
Y_test2 = np.zeros((271,))

ct1 = 0
ct2 = 0
ct3 = 0
for it in range(0, i):
    target_index = season_data_cleaned[it][0]
    if season_data_cleaned[it][1] < 19:
        seasons_10to18[ct1] = season_data_cleaned[it]
        Y_train[ct1] = match_target_scores[it][1]
        ct1 = ct1 + 1
    elif season_data_cleaned[it][1] == 19:
        season_19[ct2] = season_data_cleaned[it]
        Y_test1[ct2] = match_target_scores[it][1]
        ct2 = ct2 + 1
        # print(it)
    elif season_data_cleaned[it][1] == 20:
        season_20[ct3] = season_data_cleaned[it]
        Y_test2[ct3] = match_target_scores[it][1]
        ct3 = ct3 + 1
        # print(it)
    else:
        print('THIS SHOULD NEVER HAPPEN')

In [19]:
print(seasons_10to18.shape)
print(season_19.shape)
print(season_20.shape)

(3419, 102)
(380, 102)
(271, 102)


In [25]:
X_train = seasons_10to18
X_test1 = season_19
X_test2 = season_20



In [27]:
#model 1: Perceptron

perc_clf = Perceptron(tol=1e-3, random_state=13)
perc_clf.fit(X_train, Y_train)
Perceptron()



ValueError: Input X contains NaN.
Perceptron does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
perc_clf.score(X_train, Y_train)